In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from glob import glob
from tqdm import tqdm
from torchvision.models import resnet34, ResNet34_Weights

# 환경 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 리스트 정리
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, random_state=42)



In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from glob import glob
from tqdm import tqdm

# 환경 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 리스트 정리
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
train_files, test_files, train_labels, test_labels = train_test_split(files, labels, test_size=0.2, random_state=42)

# 데이터셋 클래스
class CTBinaryClassificationDataset(Dataset):
    def __init__(self, file_list, label_list):
        self.files = file_list
        self.labels = label_list

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = np.load(self.files[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0).astype(np.float32)
        label = self.labels[idx]
        return torch.tensor(img), torch.tensor(label).long()

# CBAM 모듈
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_mlp = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.shared_mlp(self.avg_pool(x))
        max_out = self.shared_mlp(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

# BasicBlock with CBAM
class BasicBlockWithCBAM(nn.Module):
    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.cbam = CBAM(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.cbam(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

# ResNet18 with CBAM
class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockWithCBAM(self.in_planes, planes, stride, downsample)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockWithCBAM(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# 학습 루프
train_dataset = CTBinaryClassificationDataset(train_files, train_labels)
test_dataset = CTBinaryClassificationDataset(test_files, test_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 설정
model = ResNet18_CBAM(num_classes=2).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 반복문
train_loss_list = []    # train loss를 저장할 list
test_loss_list = []     # test loss를 저장할 list

train_acc_list = []     # train 정확도 저장할 list
test_acc_list = []      # test 정확도 저장할 list

total_step = len(train_loader)  # batch_size = 16, train total 샘플 수

for epoch in range(NUM_EPOCHS):
    model.train()
    
    correct = 0
    total = 0
    epoch_loss = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        total += labels.size(0)

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}] Step [{i+1}/{total_step}], Loss: {loss.item():.4f}, Accuracy: {(correct/total)*100:.2f}')

    train_loss = epoch_loss / total_step
    train_acc = (correct / total) * 100

    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}] Done, Loss: {epoch_loss/total_step:.4f}, Accuracy: {(correct/total)*100:.2f}')

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    model.eval()

    correct = 0
    total = 0
    test_loss = 0.0  # ✅ 새로 초기화해야 함

    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)  # ✨ 배치 전체의 loss 합산
            total += labels.size(0)

            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()

            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

        test_loss_avg = test_loss / total  # ✨ 전체 sample 수로 나눔
        test_acc = correct / total * 100

        print(f'Test Loss: {test_loss_avg:.4f}, Test Accuracy: {test_acc:.2f}')

        test_loss_list.append(test_loss_avg)
        test_acc_list.append(test_acc)

    save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
    os.makedirs(save_dir, exist_ok=True)

    best_test_acc = 0.0

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        save_path = os.path.join(save_dir, "best_model_resnet18_CBAM.pth")
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")

# --- 리포트 출력 ---
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

cuda
Epoch [1/100] Step [100/267], Loss: 0.7556, Accuracy: 67.69
Epoch [1/100] Step [200/267], Loss: 0.4023, Accuracy: 68.75
Epoch [1/100] Done, Loss: 0.5918, Accuracy: 69.64
Test Loss: 0.6297, Test Accuracy: 68.51
✅ Best model saved!
Epoch [2/100] Step [100/267], Loss: 0.8439, Accuracy: 77.25
Epoch [2/100] Step [200/267], Loss: 0.4959, Accuracy: 78.66
Epoch [2/100] Done, Loss: 0.4488, Accuracy: 79.32
Test Loss: 0.5813, Test Accuracy: 72.35
✅ Best model saved!
Epoch [3/100] Step [100/267], Loss: 0.2602, Accuracy: 88.94
Epoch [3/100] Step [200/267], Loss: 0.7221, Accuracy: 88.72
Epoch [3/100] Done, Loss: 0.2779, Accuracy: 88.93
Test Loss: 0.4131, Test Accuracy: 81.63
✅ Best model saved!
Epoch [4/100] Step [100/267], Loss: 0.3009, Accuracy: 93.38
Epoch [4/100] Step [200/267], Loss: 0.2111, Accuracy: 93.16
Epoch [4/100] Done, Loss: 0.1757, Accuracy: 93.39
Test Loss: 0.4415, Test Accuracy: 83.41
✅ Best model saved!
Epoch [5/100] Step [100/267], Loss: 0.0479, Accuracy: 95.75
Epoch [5/100] S